In [ ]:

!pip install rich jedi 'lightning'
!pip install transformers tensorboardX thulac pyyaml


In [1]:

import sys
sys.path.append('.')

In [3]:
MODEL='20230428_train_gpt2_eman'

In [4]:

from utils import HParam
hp = HParam('config/train_gpt2.yaml')

In [5]:

from gpt2_generator import *
from training.util import setup_data_from_args
gpt2_model = gpt2_model_gpt2_generator(hp.gpt2.pretrained_file)
#data
data, tokenizer= setup_data_from_args(hp)

gpt2_litmodel = GPT2_BaseLitModel

if hp.kg.load_checkpoint != 'None':
    gpt2_litmodel = gpt2_litmodel.load_from_checkpoint(hp.litmodel.load_checkpoint, args=hp.gpt2, model=gpt2_model)
else:
    gpt2_litmodel = gpt2_litmodel(args=hp.gpt2, model=gpt2_model)

/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading GPT2 Model


/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [6]:

from pathlib import Path
from pytorch_lightning.utilities.rank_zero import rank_zero_info, rank_zero_only
@rank_zero_only
def _ensure_logging_dir(experiment_dir):
    """Create the logging directory via the rank-zero process, if necessary."""
    Path(experiment_dir).mkdir(parents=True, exist_ok=True)

In [7]:
import pytorch_lightning as pl

# Call baks
log_dir = Path("training_logs") / str(MODEL)
_ensure_logging_dir(log_dir)
logger = pl.loggers.TensorBoardLogger(log_dir)
experiment_dir = logger.log_dir

goldstar_metric = "validation/cer" if hp.gpt2.loss in ("transformer",) else "validation/loss"
filename_format = str(MODEL) + "epoch={epoch:04d}-validation.loss={validation/loss:.3f}"
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=2,
    filename=filename_format,
    monitor=goldstar_metric,
    mode="min",
    auto_insert_metric_name=False,
    dirpath=experiment_dir,
    every_n_epochs=hp.trainer.chkpt_every_n_epochs,
)

summary_callback = pl.callbacks.ModelSummary(max_depth=2)

callbacks = [summary_callback, checkpoint_callback]

Missing logger folder: training_logs/20230428_train_gpt2_eman/lightning_logs


In [8]:

trainer = pl.Trainer(devices=hp.trainer.devices,accelerator=hp.trainer.accelerator,
                    max_epochs=hp.trainer.max_epochs,
                    callbacks=callbacks,
                    logger = logger)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


rocm set

In [13]:

import torch
torch.set_float32_matmul_precision('high')
import os
os.environ["HSA_OVERRIDE_GFX_VERSION"]="10.3.0"

In [9]:


trainer.fit(gpt2_litmodel, datamodule=data)

You are using a CUDA device ('AMD Radeon RX 6800M') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 1/1 [00:00<00:00, 26.86it/s]


read CSV finish
demo data {'text': '16分;;E7区域的蓝方步兵被消灭，D8区域红方支援兵站被蓝方包围，;D8兵站周围发现布莱德利装甲车一辆，H9出现敌方直升机一辆，方向向东北', 'input_ids': tensor([[ 101, 8121, 1146,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]]), 'label': tensor([[ 101,  722, 1184,  ...,    0,    0,    0]])}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name              | Type            | Params
------------------------------------------------------
0 | model             | GPT2LMHeadModel | 102 M 
1 | model.transformer | GPT2Model       | 102 M 
2 | model.lm_head     | Linear          | 16.2 M
------------------------------------------------------
102 M     Trainable params
0         Non-trainable params
102 M     Total params
408.275   Total estimated model params size (MB)


/home/seafood/anaconda3/envs/rocm/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 49: 100%|██████████| 9/9 [00:05<00:00,  1.64it/s, v_num=0, validation/loss=1.660]